In [1]:
# pip install llama-index-llms-openai-like
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.llms import ChatMessage

from dotenv import load_dotenv
load_dotenv() #loads `OPENAI_API_KEY`

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/thomas/Desktop/repos/LL-ai-orchestration-with-
[nltk_data]     langchain-and-llamaindex/.venv/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
prompt = ChatMessage(
    role="user",
    content="""Write a weather report for a random city
        in ten words or less.
        Do not include any additional explanation.
""",
)

guided_prompt = ChatMessage(
    role="user",
    content=prompt.content
    + """
Return the result as JSON as follows:
{ "city": "<CITY_NAME>",
"report": "<SHORT_REPORT>" }
""",
)

In [3]:
chat = OpenAILike(
    is_chat_model=True,
    is_function_calling_model=True,
    # api_base="http://localhost:1234/v1",
    temperature=0.7,
    max_tokens=500,
    model="gpt-4-1106-preview",
)

In [4]:
def baseline():
    print("baseline:")
    print(chat.chat([prompt]))

In [5]:
def with_guided_prompt():
    print("1. Ask nicely")
    print(chat.chat([guided_prompt]))

In [6]:
def with_openai_pydantic():
    print("2. OpenAIPydandicProgram")
    from pydantic import BaseModel, Field
    from llama_index.program.openai import OpenAIPydanticProgram

    class WeatherReport(BaseModel):
        "A concise weather report for a single city"

        city: str = Field(description="City name")
        report: str = Field(description="Brief weather report")

    program = OpenAIPydanticProgram.from_defaults(
        llm=chat,
        output_cls=WeatherReport,
        prompt_template_str=prompt.content,
        verbose=True,
    )
    print(guided_prompt.content)
    py_obj = program()
    # Now a standard python obj
    # print(py_obj.city, py_obj.report)
    print(py_obj)

In [7]:
baseline()

baseline:
assistant: Toronto: Cloudy skies, chilly winds, high of 5°C, evening showers.


In [8]:
with_guided_prompt()

1. Ask nicely
assistant: ```json
{
  "city": "Reykjavik",
  "report": "Cold, windy with scattered snow showers."
}
```


In [9]:
with_openai_pydantic()

2. OpenAIPydandicProgram
Write a weather report for a random city
        in ten words or less.
        Do not include any additional explanation.

Return the result as JSON as follows:
{ "city": "<CITY_NAME>",
"report": "<SHORT_REPORT>" }

Function call: WeatherReport with args: {"city":"Reykjavik","report":"Chilly winds, scattered clouds, occasional drizzle, dress warmly."}
city='Reykjavik' report='Chilly winds, scattered clouds, occasional drizzle, dress warmly.'
